### Import Required Libraries and Set Up Environment Variables

In [343]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [338]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [342]:
#----------------- DEBUGGING ONLY -------------
#print(f'nyt_api_key:{nyt_api_key}, tmdb_api_key:{tmdb_api_key}')

#Verify we have the expected keys without exposing the data.  If data was returned this should be a 'str' class. 
print(f'Type of NYT_API_KEY {type(nyt_api_key)}')
print(f'Type of TMDB_API_KEY {type(tmdb_api_key)}')

Type of NYT_API_KEY <class 'str'>
Type of TMDB_API_KEY <class 'str'>


### Access the New York Times API

In [344]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

#----------------------START-------------------------
#Testing the hard coded API call
"""
# ----------- USING FOR DEBUGGING AND TESTING EXCEPTION CONDITIONS ONLY -----------
badUrl = url + '&fq=' + filter_query \
            + '&sort=' + sort\
            + '&fl=' + field_list\
            + '&begin_date=' + begin_date\
            + '&end_date=' + end_date
"""
# Build the base url for the NYT API query
url = url + '&api-key='+ nyt_api_key\
            + '&fq=' + filter_query \
            + '&sort=' + sort\
            + '&fl=' + field_list\
            + '&begin_date=' + begin_date\
            + '&end_date=' + end_date


#-------- Testing/Debugging only-----------------
#print(url)
#print(badUrl)

In [345]:
#response = requests.get(url)
#print(response.status_code)

In [347]:
# Create an empty list to store the reviews
reviews_list = []

#Seconds to pause in between page requests
pause_time_sec = 12

#The total pages to retrieve from 0 based index
# Per homework spec this should return 0-19 pages
pages_to_retrieve = 20


# loop through pages 0-19
for x in range(pages_to_retrieve):

    # create query with a page number
    # API results show 10 articles at a time
    query_url = url + "&page=" + str(x)

    #Use these to test error conditions/exception processing
    # bad URL assumes missing API key.
    #query_url = badUrl + "&page=" + str(x)
    # URL should return 0 results for the page as results exceed the data (currently)
    #query_url = url + "&page=" + str(200)
    
    try:
        # Make a "GET" request and retrieve the JSON
        #print('----------- EXECUTE GET ON URL ---------------')
        #print(f'GET:{query_url}');

        post_response = requests.get(query_url)
        print(f'----> GET API Execution for page={x} RESPONSE CODE:{post_response.status_code}')

        #Exception if we return status in the 400-599 range. 
        post_response.raise_for_status()

        #Retrieve the json for the request
        reviews_json = post_response.json()  
        
        # Try and save the reviews to the reviews_list        
        # loop through the reviews["response"]["docs"] and append each review to the list
        # ------------- SEE HOMEWORK INSTRUCTIONS: THIS SECTION NOT COMPLETE -----------------

        # Based on testing the json should return ['response']['docs'] even if the page has no items
        # Otherwise the returned result would have a status that would trigger and exception
        # and get picked up by the .raise_for_status() call
        # We should be able to assume this gives us valid json if no exception was raised. 
        temp_list = reviews_json['response']['docs']

        # if the json for the docs contains data we should iterate through and append it to the list.
        if len(temp_list) > 0:
            print(f'----> Appending {len(temp_list)} results for page {x}')
            for doc in temp_list:
                print(f"Append:{doc['web_url']}")
                reviews_list.append(doc)
        
            # Print the page that was just retrieved
            print(f"----> Results successfully processed for page {x}.")

            # Add a twelve second interval between queries to stay within API query limits 
            # if we are on the last page to process, don't bother sleeping execution
            if x < pages_to_retrieve - 1:
                print(f'----> API request throttling, pause for {pause_time_sec} seconds')
                time.sleep(pause_time_sec)
            else:
                print(f'----- Processed all {x} pages.  API requests complete---------')

        # if the json does not contain data, the results were empty so the page didn't return data
        # Print the page number and stop any further processing
        else:
            # Print the page number that had no results then break from the loop
            print(f'Page {x} had no results.  API requests complete')
            break;
        
    except requests.exceptions.HTTPError as e:
        print('Unexpected Request error has occurred, no further API calls will be processed')
        print(f'HttpError. {e}')
        break
    except requests.exceptions.ReadTimeout as e:
        print('Unexpected Request error has occurred, no further API calls will be processed')
        print(f'Request timed out. {e}')
        break
    except requests.exceptions.ConnectionError as e:
        print('Unexpected Request error has occurred, no further API calls will be processed')
        print(f'Connection error. {e}')
        break
    except requests.exceptions.RequestException as e:
        print('Unexpected Request error has occurred, no further API calls will be processed')
        print(f'Request exception. {e}')
        break



----> GET API Execution for page=0 RESPONSE CODE:200
------------Appending 10 results for page 0--------------
Append:https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html
Append:https://www.nytimes.com/2023/05/04/movies/whats-love-got-to-do-with-it-review.html
Append:https://www.nytimes.com/2023/05/04/movies/you-can-live-forever-review.html
Append:https://www.nytimes.com/2023/04/21/movies/a-tourists-guide-to-love-review.html
Append:https://www.nytimes.com/2023/04/20/movies/other-peoples-children-review.html
Append:https://www.nytimes.com/2023/04/13/movies/one-true-loves-movie-review.html
Append:https://www.nytimes.com/2023/04/13/movies/the-lost-weekend-a-love-story-review.html
Append:https://www.nytimes.com/2023/03/30/movies/a-thousand-and-one-review.html
Append:https://www.nytimes.com/2023/02/09/movies/your-place-or-mine-review.html
Append:https://www.nytimes.com/2023/02/02/movies/love-in-the-time-of-fentanyl-review.html
----> Results successfully processed for

In [349]:
#--------- VERIFICATION WE AT LEAST HAVE THE NUMBER OF RESPONSES EXPECTED ----------
# We have requested 20 pages with 10 items of data = 200 expected responses based on current data returned
# note this could have always changed if the data from the API is different and data was removed to have 
# less than 200 records.

reviews_list
len(reviews_list)


200

In [350]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# Grab the first 5 result from the reviews_list and dump the json data
for x in range(5):
    print(json.dumps(reviews_list[x], indent=4))


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [351]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
# json_normalize is necessary due to the nested structure for columns such as keywords which contains a nested list

review_df = pd.json_normalize(reviews_list)
review_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [352]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# Note lambda was provided by homework instruction.

review_df['title'] = review_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
review_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [354]:
#Just debugging/checking we have the new title column with the expect 200 items
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  200 non-null    object
 1   snippet                  200 non-null    object
 2   source                   200 non-null    object
 3   keywords                 200 non-null    object
 4   pub_date                 200 non-null    object
 5   word_count               200 non-null    int64 
 6   headline.main            200 non-null    object
 7   headline.kicker          47 non-null     object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  199 non-null    object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null      object
 13  byline.original          200 non-null    object
 14  byline.person            200 non-null    o

In [355]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
# Caution when debugging in Jupyter Lab - can't run the extract_keywords function twice without modifying the function
# to check the input is a list as the result is returned as a string vs. a list, we did not do this since this was
# a provided function and will assume that prior steps would be re-run for debugging as necessary.

review_df['keywords'] = review_df['keywords'].apply(extract_keywords).astype('string')
review_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [356]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

title_list = review_df['title'].to_list();
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [216]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [217]:
#-----TO DO:--------TESTING THE URL/API KEY: REMOVE BEFORE SUBMISSION -------------
print(f'base url:{url}movietitlehere{tmdb_key_string}')

base url:https://api.themoviedb.org/3/search/movie?query=movietitlehere&api_key=22867c7683e9342133f5e367de99bee0


In [221]:

def create_movie_data(json):
    genres = json['genres']
    spoken_languages = json['spoken_languages']
    production_countries = json['production_countries']
    movie_data = {}
    movie_data['title'] = json['title']
    movie_data['original_title'] = json['original_title']
    movie_data['budget'] = json['budget']
    movie_data['original_language'] = json['original_language']
    movie_data['homepage'] = json['homepage']
    movie_data['overview'] = json['overview']
    movie_data['popularity'] = json['popularity']
    movie_data['runtime'] = json['runtime']
    movie_data['revenue'] = json['revenue']
    movie_data['release_date'] = json['release_date']
    movie_data['vote_average'] = json['vote_average']
    movie_data['vote_count'] = json['vote_count']
    movie_data['genre'] = genres
    movie_data['spoken_languages'] = spoken_languages
    movie_data['production_countries'] = production_countries
    #print(movie_data)
    return movie_data

def get_movie_data(movie_id):
    try:
        # Execute "GET" request with url
        url_movie_details = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"?"+tmdb_key_string
        #print(f'Movie Details Request: {url_movie_details}')

        movie_id_response = requests.get(url_movie_details)
        #print('--------------Movie IDbRequest status-------------')
        #print(movie_id_response.status_code)

        #print('--------------Movie ID JSON-------------')
        movie_id_response_json = movie_id_response.json()
        return movie_id_response_json
        
    except requests.exceptions.HTTPError as e:
        print(f'HttpError. {e}')
    except requests.exceptions.ReadTimeout as e:
        print(f'Request timed out. {e}')
    except requests.exceptions.ConnectionError as e:
        print(f'Connection error. {e}')

def retrieve_movie_from_search(movie_search_term):
    try:
        temp_url = url + movie_search_term + tmdb_key_string
        #print(f'URL: {temp_url}')
        search_response = requests.get(temp_url)
        #print('--------------Request status-------------')
        #print(search_response.status_code)

        #print('--------------JSON-------------')
        search_response_json = search_response.json()
        #print(search_response_json)
        
        return search_response_json
        
    except requests.exceptions.HTTPError as e:
        print(f'HttpError. {e}')
    except requests.exceptions.ReadTimeout as e:
        print(f'Request timed out. {e}')
    except requests.exceptions.ConnectionError as e:
        print(f'Connection error. {e}')
    

# Create an empty list to store the results

tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests

# -------- TO DO: CHANGE PAUSE COUNTS AND TIMETR PER INSTRUCTIONS -------------
request_counter = 0
request_pause_at_requests = 10
request_pause_timer_sec = 5

# Loop through the titles
for movie in title_list:
    
    # Check if we need to sleep before making a request
    if request_counter % request_pause_at_requests == 0 and request_counter != 0:
        print(f'Request count reached {request_pause_at_requests} pause for {request_pause_timer_sec} seconds')
        time.sleep(request_pause_timer_sec)

    # Add 1 to the request counter
    request_counter = request_counter + 1
    #print(f'Retrieving Movie {request_counter}:{movie}')   
            
    # Perform a "GET" request for The Movie Database
    search_response_json = retrieve_movie_from_search(movie)
        
    # Make a request for a the full movie details    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie is not found.

        
    try:
        if len(search_response_json['results']) == 0:
            raise Exception (movie)

        # Get movie id
        #print('--------------Results-------------')
        #print(search_response_json['results'][0]['id'])

        movie_id = search_response_json['results'][0]['id']        
        #print(f'Found {movie} ID: {movie_id}')

        # Execute "GET" request with url
        movie_id_response_json = get_movie_data(movie_id)

        # -----FOLLOWING LOGIC MOVED TO A FUNCTION FOR CLARITY------        
        # Extract the genre names into a list
        # Extract the spoken_languages' English name into a list
        # Extract the production_countries' name into a list        
        # Add the relevant data to a dictionary and
        
        movie_data = create_movie_data(movie_id_response_json)

        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(movie_data)
        
        # Print out the title that was foundd
        print(f'Found {movie}')  

       # if request_counter == 2:
       #     break
                
    except Exception as eMovie:
        print(f'Not Found: {eMovie}')

print(f'tmdb movies {len(tmdb_movies_list)}')

Found The Attachment Diaries
Not Found: What’s Love Got to Do With It?’ Probably a Lo
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Request count reached 10 pause for 5 seconds
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
tmdb movies 19


In [222]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

# TO DO: CHANGE THIS TO A 5 ------------JUST MINIMIZING FOR TESTS-----------------
for x in range(2):
    print(json.dumps(tmdb_movies_list[x], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "original_language": "es",
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 3.834,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "genre": [
        {
            "id": 18,
            "name": "Drama"
        },
        {
            "id": 9648,
            "name": "Mystery"
        },
        {
            "id": 53,
            "name": "Thriller"
        },
        {
            "id": 27,
            "name": "Horror"
        }


In [223]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",3.834,102,0,2021-10-07,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",25.894,96,15055,2023-03-24,6.600,39,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.123,96,0,2023-04-21,6.310,168,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",12.012,104,84178,2022-09-21,6.816,193,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'French', 'iso_639_1': 'fr',...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.948,100,37820,2023-04-07,6.500,79,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,22.199,95,77145,2023-04-13,6.000,2,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
6,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,12.431,116,3395595,2023-03-31,6.872,90,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
7,Your Place or Mine,Your Place or Mine,0,en,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,19.836,109,0,2023-02-10,6.290,734,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,,An intimate portrait of a community fighting t...,1.607,85,0,2023-02-03,0.000,0,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
9,"Pamela, A Love Story","Pamela, A Love Story",0,en,https://www.netflix.com/title/81590934,"In her own words, through personal video and d...",16.715,113,0,2023-01-30,7.000,209,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."


### Merge and Clean the Data for Export

In [225]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  20 non-null     object
 1   snippet                  20 non-null     object
 2   source                   20 non-null     object
 3   keywords                 20 non-null     string
 4   pub_date                 20 non-null     object
 5   word_count               20 non-null     int64 
 6   headline.main            20 non-null     object
 7   headline.kicker          4 non-null      object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  20 non-null     object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null      object
 13  byline.original          20 non-null     object
 14  byline.person            20 non-null     obj

In [226]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 19 non-null     object 
 1   original_title        19 non-null     object 
 2   budget                19 non-null     int64  
 3   original_language     19 non-null     object 
 4   homepage              19 non-null     object 
 5   overview              19 non-null     object 
 6   popularity            19 non-null     float64
 7   runtime               19 non-null     int64  
 8   revenue               19 non-null     int64  
 9   release_date          19 non-null     object 
 10  vote_average          19 non-null     float64
 11  vote_count            19 non-null     int64  
 12  genre                 19 non-null     object 
 13  spoken_languages      19 non-null     object 
 14  production_countries  19 non-null     object 
dtypes: float64(2), int64(4), 

In [264]:
# Merge the New York Times reviews and TMDB DataFrames on title
#merged_apple_google = pd.merge(apple_data, google_data, on="Date")
merged_movie_df = pd.merge(review_df,tmdb_df, on='title')
merged_movie_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.834,102,0,2021-10-07,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",25.894,96,15055,2023-03-24,6.600,39,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.948,100,37820,2023-04-07,6.500,79,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,22.199,95,77145,2023-04-13,6.000,2,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.431,116,3395595,2023-03-31,6.872,90,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,When best friends and total opposites Debbie a...,19.836,109,0,2023-02-10,6.290,734,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,An intimate portrait of a community fighting t...,1.607,85,0,2023-02-03,0.000,0,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
7,https://www.nytimes.com/2023/01/19/movies/afte...,In th

In [265]:
merged_movie_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  13 non-null     object 
 1   snippet                  13 non-null     object 
 2   source                   13 non-null     object 
 3   keywords                 13 non-null     string 
 4   pub_date                 13 non-null     object 
 5   word_count               13 non-null     int64  
 6   headline.main            13 non-null     object 
 7   headline.kicker          3 non-null      object 
 8   headline.content_kicker  0 non-null      object 
 9   headline.print_headline  13 non-null     object 
 10  headline.name            0 non-null      object 
 11  headline.seo             0 non-null      object 
 12  headline.sub             0 non-null      object 
 13  byline.original          13 non-null     object 
 14  byline.person            13 

In [266]:
test_list = merged_movie_df['genre'].astype('string')
for x in test_list:
    print(x)
    x = x.replace('[','')
    x = x.replace(']','')
    x = x.replace("'",'')
    print(x)

[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}]
{id: 18, name: Drama}, {id: 9648, name: Mystery}, {id: 53, name: Thriller}, {id: 27, name: Horror}
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
{id: 18, name: Drama}, {id: 10749, name: Romance}
[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}]
{id: 10749, name: Romance}, {id: 35, name: Comedy}, {id: 18, name: Drama}
[{'id': 99, 'name': 'Documentary'}]
{id: 99, name: Documentary}
[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}]
{id: 18, name: Drama}, {id: 80, name: Crime}
[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
{id: 10749, name: Romance}, {id: 35, name: Comedy}
[{'id': 99, 'name': 'Documentary'}]
{id: 99, name: Documentary}
[{'id': 18, 'name': 'Drama'}]
{id: 18, name: Drama}
[{'id': 18, 'name': 'Drama'}]
{id: 18, name: Drama}
[{'id': 10770, 'name': 'TV Movie'}, {'id':

In [269]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
clean_columns = ['genre','spoken_languages','production_countries']

# Create a list of characters to remove
clean_chars = ["[","]","'"]

# Loop through the list of columns to fix
for column in clean_columns:
    
    # Convert the column to type 'str' 
    for x in merged_movie_df[column].astype('string'):
        # Loop through characters to remove
        for char in clean_chars:
            x = x.replace(char,'')
        merged_movie_df[column] = x

# Display the fixed DataFrame
merged_movie_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.834,102,0,2021-10-07,3.000,4,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",25.894,96,15055,2023-03-24,6.600,39,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.948,100,37820,2023-04-07,6.500,79,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,22.199,95,77145,2023-04-13,6.000,2,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.431,116,3395595,2023-03-31,6.872,90,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."


In [271]:
merged_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   web_url                  13 non-null     object 
 1   snippet                  13 non-null     object 
 2   source                   13 non-null     object 
 3   keywords                 13 non-null     string 
 4   pub_date                 13 non-null     object 
 5   word_count               13 non-null     int64  
 6   headline.main            13 non-null     object 
 7   headline.kicker          3 non-null      object 
 8   headline.content_kicker  0 non-null      object 
 9   headline.print_headline  13 non-null     object 
 10  headline.name            0 non-null      object 
 11  headline.seo             0 non-null      object 
 12  headline.sub             0 non-null      object 
 13  byline.original          13 non-null     object 
 14  byline.person            13 

In [273]:
# Drop "byline.person" column
clean_merged_movie_df = merged_movie_df.drop(columns='byline.person')
clean_merged_movie_df
#clean_merged_movie_df.info()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.834,102,0,2021-10-07,3.000,4,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",25.894,96,15055,2023-03-24,6.600,39,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.948,100,37820,2023-04-07,6.500,79,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,22.199,95,77145,2023-04-13,6.000,2,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.431,116,3395595,2023-03-31,6.872,90,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,When best friends and total opposites Debbie a...,19.836,109,0,2023-02-10,6.290,734,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,An intimate portrait of a community fighting t...,1.607,85,0,2023-02-03,0.000,0,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
7,https://www.nytimes.c

In [280]:
# Delete duplicate rows and reset index
#print(clean_merged_movie_df.duplicated(subset=['title']))
#print(clean_merged_movie_df.value_counts())
clean_movie_df = clean_merged_movie_df.drop_duplicates()
clean_movie_df.reset_index(drop=True, inplace=True)
#print(clean_movie_df.value_counts())
clean_movie_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.834,102,0,2021-10-07,3.000,4,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",25.894,96,15055,2023-03-24,6.600,39,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.948,100,37820,2023-04-07,6.500,79,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,22.199,95,77145,2023-04-13,6.000,2,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.431,116,3395595,2023-03-31,6.872,90,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
5,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,...,When best friends and total opposites Debbie a...,19.836,109,0,2023-02-10,6.290,734,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
6,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,...,An intimate portrait of a community fighting t...,1.607,85,0,2023-02-03,0.000,0,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: GB, name: United Kingdom}, {iso_3..."
7,https://www.nytimes.c

In [282]:
# Export data to CSV without the index
clean_movie_df.to_csv('test1.csv', index=False)